Showing the instance ID and uptime

In [ ]:
!hostname
!uptime
!nvidia-smi

Creating unique experiment ID to not overwrite our results every time

In [ ]:
# import sys

# import os
# from google.colab import drive
# drive.mount('/content/drive')
# %tensorflow_version 2.x
# !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
# !chmod +x Miniconda3-latest-Linux-x86_64.sh
# !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
# !time conda install -q -y -c conda-forge rdkit
# sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import datetime

# !rm -rf ./logs/
# cur_date=datetime.datetime.now().strftime("%Y%m%d-%H%M")
EXPERIMENT_ID = datetime.datetime.now().strftime("%Y%m%d-%H%M")
# EXPERIMENT_PATH = '/content/drive/My Drive/Colab Notebooks/experiments'
EXPERIMENT_PATH = 'experiments'
!mkdir -p {'%s/%s/weights' % (EXPERIMENT_PATH, EXPERIMENT_ID)}
print('EXPERIMENT_ID is: %s' % EXPERIMENT_ID)
print('Log folder is: %s/%s/logs' % (EXPERIMENT_PATH, EXPERIMENT_ID))
print('Weights folder is: %s/%s/weights' % (EXPERIMENT_PATH, EXPERIMENT_ID))
print('Model file is: %s/%s/model.h5' % (EXPERIMENT_PATH, EXPERIMENT_ID))
print('History file is: %s/%s/history.pickle' % (EXPERIMENT_PATH, EXPERIMENT_ID))

In [ ]:
import sys
import tensorflow as tf
import numpy as np
import rdkit
print("Python version: "+sys.version)
print("Numpy version: "+np.__version__)
print("Tensorflow version: "+tf.__version__)
print("Keras version: "+tf.keras.__version__)
print("RDKit version: "+rdkit.__version__)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import random

seed = 459

random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
neural_nets = ['dnn', 'scfp']

properties = [
            'ahr',
            'are',
            'ar-lbd',
            'aromatase',
            'ar',
            'atad5',
            'er-lbd',
            'er',
            'hse',
            'mmp',
            'p53',
            'ppar'
            ]

samplings = ['none', 'over', 'under']
evaluations = ['train', 'internal', 'external']
metrics = ['accuracy', 'roc_auc', 'recall', 'specificity', 'f1']

In [ ]:
from __future__ import print_function
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

# import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, GlobalMaxPooling2D, Dense, Dropout, Activation, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
TOX21_ALL_SDF_TRAIN_URL = "https://tripod.nih.gov/tox21/challenge/download?id=tox21_10k_data_allsdf"
TOX21_ALL_SDF_TEST_URL = "https://tripod.nih.gov/tox21/challenge/download?id=tox21_10k_challenge_testsdf"
tox21_all_sdf_train_file_path = tf.keras.utils.get_file("tox21_10k_data_all.sdf.zip", TOX21_ALL_SDF_TRAIN_URL, extract=True)
tox21_all_sdf_test_file_path = tf.keras.utils.get_file("tox21_10k_challenge_test.sdf.zip", TOX21_ALL_SDF_TEST_URL, extract=True)
tox21_all_sdf_train_file_path = tox21_all_sdf_train_file_path[:-4] # delete file suffix '.zip'
tox21_all_sdf_test_file_path = tox21_all_sdf_test_file_path[:-4] # delete file suffix '.zip'

In [ ]:
from tensorflow.keras import backend as K

def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity_m(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
from smiles_dnn.feature import mol_to_feature

atomsize=400

def preprocess_data(suppl, indices_raw, property):

    y = []
    errors = 0
    good_indices = []
    bad_indices = []

    for index_raw in indices_raw:
        try:
            if neural_net == 'dnn':
                fp = AllChem.GetMorganFingerprintAsBitVect(suppl[index_raw], 2, nBits=2048)
            elif neural_net == 'scfp':
                if len(Chem.MolToSmiles(suppl[index_raw], kekuleSmiles=True, isomericSmiles=True)) > atomsize:
                    raise Exception
                mol_to_feature(suppl[index_raw], -1, atomsize) # Checking wether conversion is possible

            y.append(suppl[index_raw].GetBoolProp(property))
            good_indices.append(index_raw) # If we made it this far, x and y can be generated for index_raw

        except Exception as inst:
#             print(type(inst))
#             print(inst.args)
#             print(inst)

            bad_indices.append(index_raw)
            errors+=1

    print('%i errors occured' % errors)
    
    return good_indices, y, bad_indices

In [ ]:
# SCFP architecture
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization, Activation

featuresize = 42

def get_scfp_net():
    model = Sequential()
    with tf.name_scope('Input'):
        model.add(Conv2D(filters=128, kernel_size=(1, featuresize), strides=1, input_shape=(atomsize, featuresize, 1), use_bias=False))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        
    with tf.name_scope('generate_scfp'):
        model.add(AveragePooling2D(pool_size=(5, 1), strides=1, padding='same'))
        model.add(Conv2D(filters=64, kernel_size=(1, 1), strides=1, use_bias=False))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(AveragePooling2D(pool_size=(5, 1), strides=1, padding='same'))

#         model.add(Conv1D(filters=320, kernel_size=5, strides=1, input_shape=(atomsize, featuresize), activation='relu'))
#         model.add(AveragePooling1D(pool_size=5, strides=1, padding='same'))
#         model.add(Conv1D(filters=170, kernel_size=15, strides=1, activation='relu'))
#         model.add(AveragePooling1D(pool_size=5, strides=1, padding='same'))


#     with tf.name_scope('Dropout'):
#         model.add(Dropout(0.2))
    with tf.name_scope('GlobalMaxPooling'):
        model.add(GlobalMaxPooling2D())
    with tf.name_scope('Hidden'):
        model.add(Dense(264, activation='relu'))
    with tf.name_scope('Output'):
        model.add(Dense(1, activation='sigmoid'))

    print(model.summary())
    
    return model

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler, ModelCheckpoint, EarlyStopping

def fit_scfp_model(suppl, x_train_indices, y_train, x_val_indices, y_val, hparams, callbacks):

    model = get_scfp_net()

    model.compile(
        loss=hparams[HP_LOSS_FUNCTION],
        optimizer=hparams[HP_OPTIMIZER],
        metrics=['accuracy', 'AUC', specificity_m, 'Recall', f1_score_m],
    )

    model.optimizer.lr = hparams[HP_LEARNING_RATE]
    model.optimizer.decay = hparams[HP_DECAY]
    model.optimizer.momentum = hparams[HP_MOMENTUM]
    model.optimizer.clipnorm = 1.0
        
#     generator = SCFPFeatureMatrixGenerator(x_set=x_train_indices,
#                                        y_set=y_train,
#                                        batch_size=hparams[HP_BATCH_SIZE],
#                                        suppl=suppl)

#     generator_validation = SCFPFeatureMatrixGenerator(x_set=x_val_indices,
#                                                   y_set=y_val,
#                                                   batch_size=hparams[HP_BATCH_SIZE],
#                                                   suppl=suppl)

    x_train = [ mol_to_feature(suppl[id], -1, atomsize) for id in x_train_indices ]
    x_val = [ mol_to_feature(suppl[id], -1, atomsize) for id in x_val_indices ]

    x_train = np.asarray(x_train, dtype=np.float32)
    x_val = np.asarray(x_val, dtype=np.float32)
    
    y_train = np.asarray(y_train, dtype=np.int32)
    y_val = np.asarray(y_val, dtype=np.int32)
    
    x_train = x_train.reshape(-1, atomsize, featuresize, 1)
    x_val = x_val.reshape(-1, atomsize, featuresize, 1)

    
    history = model.fit(x=x_train,
                        y=y_train,
                        validation_data=(x_val, y_val),
#                         x=generator,
#                         validation_data=generator_validation,
                        epochs=hparams[HP_EPOCHS],
#                         initial_epoch=initial_epoch,
                        callbacks=callbacks,
                        verbose=2
                       )
    
    return model, history

In [ ]:
def get_dnn(hparams, output_bias):
    model = Sequential()

    with tf.name_scope('Input'):
        model.add(Dropout(hparams[HP_DROPOUT_INPUT], input_shape=(2048,))) # HARD CODED VALUE for fingerprint dimensions
        model.add(Dense(
#             input_dim=x_train.shape[1],
            hparams[HP_NUM_UNITS],
            kernel_regularizer=l2(hparams[HP_L2_L]),
            bias_regularizer=l2(hparams[HP_L2_L])))
        model.add(BatchNormalization())
        model.add(Activation('relu'))

    with tf.name_scope('Hidden'):
        for layer in range(hparams[HP_NUM_HIDDEN_LAYERS]): # Dynamically adding hidden layers with a subsequent dropout
            model.add(Dense(
                hparams[HP_NUM_UNITS],
                kernel_regularizer=l2(hparams[HP_L2_L]),
                bias_regularizer=l2(hparams[HP_L2_L])))
            model.add(BatchNormalization())
            model.add(Activation('relu'))
            model.add(Dropout(hparams[HP_DROPOUT]))
    with tf.name_scope('Output'):
        model.add(Dense(
            1,
            kernel_regularizer=l2(hparams[HP_L2_L]),
            bias_regularizer=l2(hparams[HP_L2_L]),
            bias_initializer=output_bias))
        model.add(BatchNormalization())
        model.add(Activation('sigmoid'))
   
    return model

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

def fit_dnn_model(suppl, x_train_indices, y_train, x_val_indices, y_val, hparams, callbacks):

    negatives = np.array(y_train).flatten().tolist().count(0)
    positives = np.array(y_train).flatten().tolist().count(1)
    
    output_bias = tf.keras.initializers.Constant(np.log([positives/negatives]))
    
    model = get_dnn(hparams, output_bias)

    model.compile(
        loss=hparams[HP_LOSS_FUNCTION],
        optimizer=hparams[HP_OPTIMIZER],
        metrics=['accuracy', 'AUC', specificity_m, 'Recall', f1_score_m],
    )

    model.optimizer.nesterov = True
    model.optimizer.lr = hparams[HP_LEARNING_RATE]
    model.optimizer.decay = hparams[HP_DECAY]
    model.optimizer.momentum = hparams[HP_MOMENTUM]
    model.optimizer.clipnorm = 1.0

    x_train = np.asarray([ AllChem.GetMorganFingerprintAsBitVect(suppl[id], 2, nBits=2048) for id in x_train_indices ])
    x_val = np.asarray([ AllChem.GetMorganFingerprintAsBitVect(suppl[id], 2, nBits=2048) for id in x_val_indices ])

#     x_train = np.asarray(x_train, dtype=np.int32)
#     x_val = np.asarray(x_val, dtype=np.int32)
    
#     y_train = np.asarray(y_train, dtype=np.int32)
#     y_val = np.asarray(y_val, dtype=np.int32)
    
#     x_train = x_train.reshape(-1, atomsize, featuresize, 1)
#     x_val = x_val.reshape(-1, atomsize, featuresize, 1)

    
    history = model.fit(x=x_train,
                        y=y_train,
                        validation_data=(x_val, y_val),
#                         x=generator,
#                         validation_data=generator_validation,
                        epochs=hparams[HP_EPOCHS],
#                         initial_epoch=initial_epoch,
                        callbacks=callbacks,
                        verbose=2
                       )
    
    return model, history

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, classification_report, make_scorer, recall_score

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.optimizers import RMSprop, Adam, SGD


HP_EPOCHS = hp.HParam('epochs', hp.Discrete([250]))
# HP_EPOCHS = hp.HParam('epochs', hp.Discrete([40]))
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([32])) # We use minibatching, because it will converge faster
HP_LOSS_FUNCTION = hp.HParam('loss_function', hp.Discrete(['binary_crossentropy']))
# HP_LOSS_FUNCTION = hp.HParam('loss_function', hp.Discrete(['masked_loss_function']))
# HP_LOSS_FUNCTION = hp.HParam('loss_function', hp.Discrete(['binary_crossentropy', 'mse', 'masked_loss_function']))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['Adam']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['SGD', 'RMSProp', 'Adam']))
# HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-1]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([1e-3]))
HP_DECAY = hp.HParam('decay', hp.Discrete([1e-6]))
HP_MOMENTUM = hp.HParam('momentum', hp.Discrete([0.9]))
HP_L2_L = hp.HParam('l2_l', hp.Discrete([1e-4]))

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([1024]))
HP_NUM_HIDDEN_LAYERS = hp.HParam('num_hidden_layers', hp.Discrete([2]))
HP_DROPOUT_INPUT = hp.HParam('dropout_input', hp.Discrete([0.2]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.5]))

# properties = ['NR-AhR']
# properties = ['NR-ER-LBD']
# properties = ['NR-AhR', 'NR-ER-LBD']
# properties = ['NR-AhR', 'NR-AR', 'NR-AR-LBD', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53', 'NR-Aromatase']


results = {}

for neural_net in neural_nets:
    results[neural_net] = {}
    for property in properties:
        results[neural_net][property] = {}
        for sampling in samplings:
            results[neural_net][property][sampling] = {}
            for evaluation in evaluations:
                results[neural_net][property][sampling][evaluation] = {}
                for metric in metrics:
                    results[neural_net][property][sampling][evaluation][metric] = []

#     !echo {properties} > {"%s/%s/properties.txt" % (EXPERIMENT_PATH, EXPERIMENT_ID)}
print(properties)



k = 10

kfold = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)

for neural_net in neural_nets:
    for property in properties:

        suppl = Chem.SDMolSupplier('data/curated/%s.sdf' % property)

        x_indices, y, bad_indices = preprocess_data(suppl, range(len(suppl)), property)

    #     negatives = np.array(y).flatten().tolist().count(0)
    #     positives = np.array(y).flatten().tolist().count(1)
    #     print('Percentage of positives: %f' % (positives/negatives))

        print("Hash of indices list %s" % (hex(hash(frozenset(x_indices)))))

        # first split into training + validation (90%) and test (10%)
        x_train_val_indices, x_test_indices, y_train_val, y_test = train_test_split(x_indices, y, test_size=0.2, stratify=y, random_state=seed)

        print("Hash of indices list - train_val: %s, test: %s" % (hex(hash(frozenset(x_train_val_indices))), hex(hash(frozenset(x_test_indices)))))

    #     negatives = np.array(y_train_val).flatten().tolist().count(0)
    #     positives = np.array(y_train_val).flatten().tolist().count(1)
    #     print('Percentage of positives in train_val: %f' % (positives/negatives))

    #     negatives = np.array(y_test).flatten().tolist().count(0)
    #     positives = np.array(y_test).flatten().tolist().count(1)
    #     print('Percentage of positives in test: %f' % (positives/negatives))

        # Double checking wether our data sets are correctly disjoint
        if not set(frozenset(x_train_val_indices)).isdisjoint(frozenset(x_test_indices)):
            raise Exception

        for fold, (train_indices_of_kfold, val_indices_of_kfold) in enumerate(kfold.split(X=x_train_val_indices, y=y_train_val)):

            for sampling in samplings:
                
                print("Doing neural net %s with property %s with sampling %s in fold %i" % (neural_net, property, sampling, fold))

                x_train_indices = np.array(x_train_val_indices)[train_indices_of_kfold.astype(int)]
                x_val_indices = np.array(x_train_val_indices)[val_indices_of_kfold.astype(int)]

                y_train = np.array(y_train_val)[train_indices_of_kfold.astype(int)]
                y_val = np.array(y_train_val)[val_indices_of_kfold.astype(int)]

                # Sampling the data

                if sampling == 'over':
                    x_train_indices, y_train = RandomOverSampler().fit_sample(X=np.reshape(x_train_indices, (-1, 1)), y=y_train)
                    x_train_indices = np.reshape(x_train_indices, (-1,))

                    x_val_indices, y_val = RandomOverSampler().fit_sample(X=np.reshape(x_val_indices, (-1, 1)), y=y_val)
                    x_val_indices = np.reshape(x_val_indices, (-1,))

                    y_train = np.reshape(y_train, (-1, 1))
                    y_val = np.reshape(y_val, (-1, 1))
                elif sampling == 'under':
                    x_train_indices, y_train = RandomUnderSampler().fit_sample(X=np.reshape(x_train_indices, (-1, 1)), y=y_train)
                    x_train_indices = np.reshape(x_train_indices, (-1,))

                    x_val_indices, y_val = RandomUnderSampler().fit_sample(X=np.reshape(x_val_indices, (-1, 1)), y=y_val)
                    x_val_indices = np.reshape(x_val_indices, (-1,))

                    y_train = np.reshape(y_train, (-1, 1))
                    y_val = np.reshape(y_val, (-1, 1))


                print("Hash of indices list - train: %s, validation: %s" % (hex(hash(frozenset(x_train_indices))), hex(hash(frozenset(x_val_indices)))))

        #         negatives = np.array(y_train).flatten().tolist().count(0)
        #         positives = np.array(y_train).flatten().tolist().count(1)
        #         print('Percentage of positives in train for fold %i: %f' % (fold, positives/negatives))

        #         negatives = np.array(y_val).flatten().tolist().count(0)
        #         positives = np.array(y_val).flatten().tolist().count(1)
        #         print('Percentage of positives in val for fold %i: %f' % (fold, positives/negatives))

                # Check wether lists are really disjoint!
                if not set(frozenset(x_train_indices)).isdisjoint(frozenset(x_val_indices)):
                    raise Exception

                if not set(frozenset(x_train_indices)).isdisjoint(frozenset(x_test_indices)):
                    raise Exception

                x_train_indices = x_train_indices.tolist()
                x_val_indices = x_val_indices.tolist()

                for epochs in HP_EPOCHS.domain.values:
                    for batch_size in HP_BATCH_SIZE.domain.values:
                        for l2_l in HP_L2_L.domain.values:
                            for loss_function in HP_LOSS_FUNCTION.domain.values:
                                for optimizer in HP_OPTIMIZER.domain.values:
                                    for learning_rate in HP_LEARNING_RATE.domain.values:
                                        for decay in HP_DECAY.domain.values:
                                            for momentum in HP_MOMENTUM.domain.values:
                                                for num_units in HP_NUM_UNITS.domain.values:
                                                    for num_hidden_layers in HP_NUM_HIDDEN_LAYERS.domain.values:
                                                        for dropout_input in HP_DROPOUT_INPUT.domain.values:
                                                            for dropout_rate in HP_DROPOUT.domain.values:

                                                                hparams = {
                                                                    HP_EPOCHS: epochs,
                                                                    HP_BATCH_SIZE: batch_size,
                                                                    HP_LOSS_FUNCTION: loss_function,
                                                                    HP_OPTIMIZER: optimizer,
                                                                    HP_LEARNING_RATE: learning_rate,
                                                                    HP_DECAY: decay,
                                                                    HP_MOMENTUM: momentum,
                                                                    HP_L2_L: l2_l,
                                                                    HP_NUM_UNITS: num_units,
                                                                    HP_NUM_HIDDEN_LAYERS: num_hidden_layers,
                                                                    HP_DROPOUT_INPUT: dropout_input,
                                                                    HP_DROPOUT: dropout_rate,
                                                                }

                                                                log_dir = "%s/%s/%s-%s-%s-fold-%i/logs" % (EXPERIMENT_PATH, EXPERIMENT_ID, neural_net, property, sampling, fold)
                                                                weights_path = '%s/%s/%s-%s-%s-fold-%i/weights.h5' % (EXPERIMENT_PATH, EXPERIMENT_ID, neural_net, property, sampling, fold)

                                                                callbacks = [
                                                                    TensorBoard(log_dir=log_dir),
                                                                    hp.KerasCallback(log_dir, hparams, trial_id='%s-%s-%s-%s-%i' % (EXPERIMENT_ID, neural_net, property, sampling, fold)), # log hparams
                                                                    ModelCheckpoint(filepath=weights_path,
                            #                                                     monitor='val_f1_score_m',
                                                                                monitor='val_recall',
                                                                                verbose=0,
                                                                                save_best_only=True,
                                                                                mode='auto'),
                            #                                         EarlyStopping(monitor='val_f1_score_m',
                            #                                             verbose=0,
                            #                                             patience=50,
                            #                                             mode='max',
                            #                                             restore_best_weights=True),
                                                                                ]

                                                                if neural_net == 'dnn':
                                                                    model, history = fit_dnn_model(suppl, x_train_indices, y_train, x_val_indices, y_val, hparams, callbacks)
                                                                elif neural_net == 'scfp':                                    
                                                                    model, history = fit_scfp_model(suppl, x_train_indices, y_train, x_val_indices, y_val, hparams, callbacks)

                                                                model.save('%s/%s/%s-%s-%s-fold-%i/model.h5' % (EXPERIMENT_PATH, EXPERIMENT_ID, neural_net, property, sampling, fold))

                                                                for evaluation in ['train', 'internal']:
                                                                    if evaluation == 'train':
                                                                        x_indices = x_train_indices
                                                                        y = y_train

                                                                    elif evaluation == 'internal':
                                                                        x_indices = x_val_indices
                                                                        y = y_val

                                                                    if neural_net == 'dnn':
                                                                        x = [ AllChem.GetMorganFingerprintAsBitVect(suppl[id], 2, nBits=2048) for id in x_indices ]
                                                                        x = np.asarray(x, dtype=np.float32)
                                                                    elif neural_net == 'scfp':
                                                                        x = [ mol_to_feature(suppl[id], -1, atomsize) for id in x_indices ]
                                                                        x = np.asarray(x, dtype=np.float32)
                                                                        x = x.reshape(-1, atomsize, featuresize, 1)

                                                                    y_pred = model.predict(x)

                                                                    results[neural_net][property][sampling][evaluation]['accuracy'].append(accuracy_score(y, y_pred.round()))
                                                                    results[neural_net][property][sampling][evaluation]['roc_auc'].append(roc_auc_score(y, y_pred))
                                                                    results[neural_net][property][sampling][evaluation]['recall'].append(recall_score(y, y_pred.round(), pos_label=1))
                                                                    results[neural_net][property][sampling][evaluation]['specificity'].append(recall_score(y, y_pred.round(), pos_label=0))
                                                                    results[neural_net][property][sampling][evaluation]['f1'].append(f1_score(y, y_pred.round()))

Saving results in pickle

In [ ]:
import pickle

# Load random forest results
    
# EXPERIMENT_PATH='experiments-rf'
# EXPERIMENT_ID='20201023-1839'

# results_pickle_filename = '%s/%s/results.pickle' % (EXPERIMENT_PATH, EXPERIMENT_ID)
    
# # # Loading from file
# with open(results_pickle_filename, "rb") as results_pickle_file:
#     results_rf = pickle.load(results_pickle_file)

# EXPERIMENT_PATH='experiments'
# EXPERIMENT_ID='20200904-2220'

results_pickle_filename = '%s/%s/results.pickle' % (EXPERIMENT_PATH, EXPERIMENT_ID)

# Writing to file
with open(results_pickle_filename, "wb") as results_pickle_file:
    pickle.dump(results, results_pickle_file) 

# Loading from file
# with open(results_pickle_filename, "rb") as results_pickle_file:
#     results = pickle.load(results_pickle_file)

Evaluating best model in regards to our Single Number Evaluation Metric

In [ ]:
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, f1_score, classification_report, make_scorer, recall_score
from sklearn.model_selection import train_test_split

SNEM = 'f1'

dependencies = {
    'specificity_m': specificity_m,
    'f1_score_m': f1_score_m,
}

for neural_net in neural_nets:
    for property in properties:
        
        suppl = Chem.SDMolSupplier('data/curated/%s.sdf' % property)

        x_indices, y, bad_indices = preprocess_data(suppl, range(len(suppl)), property)

        print("Hash of indices list %s" % (hex(hash(frozenset(x_indices)))))

        # first split into training + validation (90%) and test (10%)
        x_train_val_indices, x_test_indices, y_train_val, y_test = train_test_split(x_indices, y, test_size=0.2, stratify=y, random_state=seed)

        print("Hash of indices list - train_val: %s, test: %s" % (hex(hash(frozenset(x_train_val_indices))), hex(hash(frozenset(x_test_indices)))))
        
        negatives_full = np.array(y).flatten().tolist().count(0)
        positives_full = np.array(y).flatten().tolist().count(1)
        print('Entire dataset - Property: %s, size: %i, positives: %i, negatives: %i, percentage: %f' % (property, len(y), positives_full, negatives_full, (positives_full/negatives_full)))

        negatives_train = np.array(y_train_val).flatten().tolist().count(0)
        positives_train = np.array(y_train_val).flatten().tolist().count(1)
        print('Train dataset - Property: %s, size: %i, positives: %i, negatives: %i, percentage: %f' % (property, len(y_train_val), positives_train, negatives_train, (positives_train/negatives_train)))

        negatives_test = np.array(y_test).flatten().tolist().count(0)
        positives_test = np.array(y_test).flatten().tolist().count(1)
        print('Test dataset - Property: %s, size: %i, positives: %i, negatives: %i, percentage: %f' % (property, len(y_test), positives_test, negatives_test, (positives_test/negatives_test)))
        
        print('%s & %i & %f & %i & %f & %i & %f' % (
                                                    property, len(y),
                                                    round(positives_full/negatives_full, 3),
                                                    len(y_train_val),
                                                    round(positives_train/negatives_train, 3),
                                                    len(y_test),
                                                    round(positives_test/negatives_test, 3)
                                                    )
             )
        
        # Double checking wether our data sets are correctly disjoint
        if not set(frozenset(x_train_val_indices)).isdisjoint(frozenset(x_test_indices)):
            raise Exception
        
        for sampling in samplings:

            best_model_index = results[neural_net][property][sampling]['internal'][SNEM].index(max(results[neural_net][property][sampling]['internal'][SNEM]))
            
            best_model = load_model('%s/%s/%s-%s-%s-fold-%i/model.h5' % (EXPERIMENT_PATH, EXPERIMENT_ID, neural_net, property, sampling, best_model_index), custom_objects=dependencies)

            if neural_net == 'dnn':
                x = [ AllChem.GetMorganFingerprintAsBitVect(suppl[id], 2, nBits=2048) for id in x_test_indices ]
                x = np.asarray(x, dtype=np.float32)
            elif neural_net == 'scfp':
                x = [ mol_to_feature(suppl[id], -1, atomsize) for id in x_test_indices ]
                x = np.asarray(x, dtype=np.float32)
                x = x.reshape(-1, atomsize, featuresize, 1)


        #     x = [ mol_to_feature(suppl[index], -1, atomsize) for index in x_test_indices ]
        #     x = np.asarray(x, dtype=np.float32)
        #     x = x.reshape(-1, atomsize, featuresize, 1)

            y_pred = best_model.predict(x)


            results[neural_net][property][sampling]['external']['accuracy'] = accuracy_score(y_test, y_pred.round())
            results[neural_net][property][sampling]['external']['roc_auc'] = roc_auc_score(y_test, y_pred)
            results[neural_net][property][sampling]['external']['recall'] = recall_score(y_test, y_pred.round(), pos_label=1)
            results[neural_net][property][sampling]['external']['specificity'] = recall_score(y_test, y_pred.round(), pos_label=0)
            results[neural_net][property][sampling]['external']['f1'] = f1_score(y_test, y_pred.round())

            print('External evaluation of %s for dataset: %s with sampling: %s\nAccuracy:\t%f\nf1-score:\t%f\nRecall:\t%f\nAUC:\t%f\nSpecificity:\t%f'
              % (neural_net,
                 property,
                 sampling,
                 results[neural_net][property][sampling]['external']['accuracy'],
                 results[neural_net][property][sampling]['external']['f1'],
                 results[neural_net][property][sampling]['external']['recall'],
                 results[neural_net][property][sampling]['external']['roc_auc'],
                 results[neural_net][property][sampling]['external']['specificity'],
                 ))

Printing out results and creating bar chart with error bar

In [ ]:
import matplotlib.pyplot as plt

import datetime
EXPERIMENT_ID = datetime.datetime.now().strftime("%Y%m%d-%H%M")

!mkdir -p {'%s/%s' % (EXPERIMENT_PATH, EXPERIMENT_ID)}

# neural_nets = ['dnn', 'scfp']

# properties = [
#             'ahr',
#             'are',
#             'ar-lbd',
#             'aromatase',
#             'ar',
#             'atad5',
#             'er-lbd',
#             'er',
#             'hse',
#             'mmp',
#             'p53',
#             'ppar'
#             ]

# samplings = ['none', 'over', 'under']
# evaluations = ['train', 'test', 'train_std', 'test_std', 'external']
# metrics = ['accuracy', 'roc_auc', 'recall', 'specificity', 'f1']

for property in properties:
    for sampling in samplings:
        for neural_net in neural_nets:
            print('Average scores of %s on train data for dataset: %s with sampling: %s\nAccuracy:\t%f\nf1-score:\t%f\nRecall:\t%f\nAUC:\t%f\nSpecifity:\t%f'
              % (neural_net,
                 property,
                 sampling,
                 np.mean(results[neural_net][property][sampling]['train']['accuracy']),
                 np.mean(results[neural_net][property][sampling]['train']['f1']),
                 np.mean(results[neural_net][property][sampling]['train']['recall']),
                 np.mean(results[neural_net][property][sampling]['train']['roc_auc']),
                 np.mean(results[neural_net][property][sampling]['train']['specificity']),
                 ))
            print()
            print('Standard deviations of %s on train data for dataset: %s with sampling: %s\nAccuracy:\t%f\nf1-score:\t%f\nRecall:\t%f\nAUC:\t%f\nSpecifity:\t%f'
              % (neural_net,
                 property,
                 sampling,
                 np.std(results[neural_net][property][sampling]['train']['accuracy']),
                 np.std(results[neural_net][property][sampling]['train']['f1']),
                 np.std(results[neural_net][property][sampling]['train']['recall']),
                 np.std(results[neural_net][property][sampling]['train']['roc_auc']),
                 np.std(results[neural_net][property][sampling]['train']['specificity']),
                 ))
            print()
            print('Average scores of %s on cross validation for dataset: %s with sampling: %s\nAccuracy:\t%f\nf1-score:\t%f\nRecall:\t%f\nAUC:\t%f\nSpecificity:\t%f'
              % (neural_net,
                 property,
                 sampling,
                 np.mean(results[neural_net][property][sampling]['internal']['accuracy']),
                 np.mean(results[neural_net][property][sampling]['internal']['f1']),
                 np.mean(results[neural_net][property][sampling]['internal']['recall']),
                 np.mean(results[neural_net][property][sampling]['internal']['roc_auc']),
                 np.mean(results[neural_net][property][sampling]['internal']['specificity']),
                 ))
            print()
            print('Standard deviations of %s on cross validation for dataset: %s with sampling: %s\nAccuracy:\t%f\nf1-score:\t%f\nRecall:\t%f\nAUC:\t%f\nSpecificity:\t%f'
              % (neural_net,
                 property,
                 sampling,
                 np.std(results[neural_net][property][sampling]['internal']['accuracy']),
                 np.std(results[neural_net][property][sampling]['internal']['f1']),
                 np.std(results[neural_net][property][sampling]['internal']['recall']),
                 np.std(results[neural_net][property][sampling]['internal']['roc_auc']),
                 np.std(results[neural_net][property][sampling]['internal']['specificity']),
                 ))
            print()
            print('Best models of %s in internal evaluation for dataset: %s with sampling: %s\nAccuracy:\t%i\nf1-score:\t%i\nRecall:\t%i\nAUC:\t%i\nSpecificity:\t%i'
              % (neural_net,
                 property,
                 sampling,
                 results[neural_net][property][sampling]['internal']['accuracy'].index(max(results[neural_net][property][sampling]['internal']['accuracy'])),
                 results[neural_net][property][sampling]['internal']['f1'].index(max(results[neural_net][property][sampling]['internal']['f1'])),
                 results[neural_net][property][sampling]['internal']['recall'].index(max(results[neural_net][property][sampling]['internal']['recall'])),
                 results[neural_net][property][sampling]['internal']['roc_auc'].index(max(results[neural_net][property][sampling]['internal']['roc_auc'])),
                 results[neural_net][property][sampling]['internal']['specificity'].index(max(results[neural_net][property][sampling]['internal']['specificity'])),
                 ))
            print()
            print('Best metrics of %s in internal evaluation for dataset: %s with sampling: %s\nAccuracy:\t%f\nf1-score:\t%f\nRecall:\t%f\nAUC:\t%f\nSpecificity:\t%f'
              % (neural_net,
                 property,
                 sampling,
                 max(results[neural_net][property][sampling]['internal']['accuracy']),
                 max(results[neural_net][property][sampling]['internal']['f1']),
                 max(results[neural_net][property][sampling]['internal']['recall']),
                 max(results[neural_net][property][sampling]['internal']['roc_auc']),
                 max(results[neural_net][property][sampling]['internal']['specificity']),
                 ))
            print()

        # Create lists for the plot
        plot_metrics = ['Accuracy', 'f1-score', 'Recall', 'AUC', 'Specifity']
        x_pos = np.arange(len(plot_metrics))

        # Creating plot for CV

        mean_dnn = [
            np.mean(results['dnn'][property][sampling]['internal']['accuracy']),
            np.mean(results['dnn'][property][sampling]['internal']['f1']),
            np.mean(results['dnn'][property][sampling]['internal']['recall']),
            np.mean(results['dnn'][property][sampling]['internal']['roc_auc']),
            np.mean(results['dnn'][property][sampling]['internal']['specificity'])
            ]

        error_dnn = [
            np.std(results['dnn'][property][sampling]['internal']['accuracy']),
            np.std(results['dnn'][property][sampling]['internal']['f1']),
            np.std(results['dnn'][property][sampling]['internal']['recall']),
            np.std(results['dnn'][property][sampling]['internal']['roc_auc']),
            np.std(results['dnn'][property][sampling]['internal']['specificity']),
            ]
        
        mean_scfp = [
            np.mean(results['scfp'][property][sampling]['internal']['accuracy']),
            np.mean(results['scfp'][property][sampling]['internal']['f1']),
            np.mean(results['scfp'][property][sampling]['internal']['recall']),
            np.mean(results['scfp'][property][sampling]['internal']['roc_auc']),
            np.mean(results['scfp'][property][sampling]['internal']['specificity'])
            ]

        error_scfp = [
            np.std(results['scfp'][property][sampling]['internal']['accuracy']),
            np.std(results['scfp'][property][sampling]['internal']['f1']),
            np.std(results['scfp'][property][sampling]['internal']['recall']),
            np.std(results['scfp'][property][sampling]['internal']['roc_auc']),
            np.std(results['scfp'][property][sampling]['internal']['specificity']),
            ]
        
        mean_rf = [
            np.mean(results_rf[property][sampling]['test']['accuracy']),
            np.mean(results_rf[property][sampling]['test']['f1']),
            np.mean(results_rf[property][sampling]['test']['recall']),
            np.mean(results_rf[property][sampling]['test']['roc_auc']),
            np.mean(results_rf[property][sampling]['test']['specificity'])
            ]

        error_rf = [
            results_rf[property][sampling]['test_std']['accuracy'],
            results_rf[property][sampling]['test_std']['f1'],
            results_rf[property][sampling]['test_std']['recall'],
            results_rf[property][sampling]['test_std']['roc_auc'],
            results_rf[property][sampling]['test_std']['specificity'],
            ]

        plt.rcParams["figure.figsize"] = (8, 7)
        plt.rcParams['text.usetex'] = True
        plt.rcParams['font.family'] = 'serif'
        plt.rcParams['font.size'] = '28'

#         color = ['pink', 'mediumaquamarine', 'lightskyblue', 'wheat', 'mediumpurple']
        width = 0.2

        fig, ax = plt.subplots()
        ax.bar(x_pos - width, mean_dnn, width, label='DNN', yerr=error_dnn, align='center', color="lightskyblue", hatch='/', ecolor='black', capsize=5)
        ax.bar(x_pos, mean_scfp, width, label='CNN', yerr=error_scfp, align='center', color="mediumaquamarine", hatch='\\', ecolor='black', capsize=5)
        ax.bar(x_pos + width, mean_rf, width, label='RF', yerr=error_rf, align='center', color="mediumpurple", hatch='', ecolor='black', capsize=5)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(plot_metrics)

        # Save the figure and show
        plt.tight_layout()
        plt.minorticks_on()
        ax.tick_params(axis='x', which='minor', bottom=False)
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=45)
        plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)
        plt.savefig('%s/%s/bar_plot_cv_%s_%s' % (EXPERIMENT_PATH, EXPERIMENT_ID, property, sampling), dpi=300, bbox_inches = "tight")
        plt.show()

        # Creating plots for external

        external_dnn = [
            np.mean(results['dnn'][property][sampling]['external']['accuracy']),
            np.mean(results['dnn'][property][sampling]['external']['f1']),
            np.mean(results['dnn'][property][sampling]['external']['recall']),
            np.mean(results['dnn'][property][sampling]['external']['roc_auc']),
            np.mean(results['dnn'][property][sampling]['external']['specificity'])
            ]

        external_scfp = [
            np.mean(results['scfp'][property][sampling]['external']['accuracy']),
            np.mean(results['scfp'][property][sampling]['external']['f1']),
            np.mean(results['scfp'][property][sampling]['external']['recall']),
            np.mean(results['scfp'][property][sampling]['external']['roc_auc']),
            np.mean(results['scfp'][property][sampling]['external']['specificity'])
            ]
        
        external_rf = [
            np.mean(results_rf[property][sampling]['external']['accuracy']),
            np.mean(results_rf[property][sampling]['external']['f1']),
            np.mean(results_rf[property][sampling]['external']['recall']),
            np.mean(results_rf[property][sampling]['external']['roc_auc']),
            np.mean(results_rf[property][sampling]['external']['specificity'])
            ]

        fig, ax = plt.subplots()
        ax.bar(x_pos - width, external_dnn, width, label='DNN', align='center', color="lightskyblue", hatch='/')
        ax.bar(x_pos, external_scfp, width, label='CNN', align='center', color="mediumaquamarine", hatch='\\')
        ax.bar(x_pos + width, external_rf, width, label='RF', align='center', color="mediumpurple", hatch='')
        ax.set_xticks(x_pos)
        ax.set_xticklabels(plot_metrics)

        # Save the figure and show
        plt.tight_layout()
        plt.minorticks_on()
        ax.tick_params(axis='x', which='minor', bottom=False)
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.xticks(rotation=45)
        plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)
        plt.savefig('%s/%s/bar_plot_external_%s_%s' % (EXPERIMENT_PATH, EXPERIMENT_ID, property, sampling), dpi=300, bbox_inches = "tight")
        plt.show()